# Notebook for "simpler" Machine Learning approaches

This notebook takes the preprocessed data and applies and evaluates different machine learning techniques for predicting future take rates.

In [1]:
import pandas as pd

## Imort dataframe prepared for machine learning (MLbase_DataFrame)

In [5]:
df = pd.read_csv('../CLAAS_data/MLbase_DataFrame.csv')

In [6]:
df

,Datum,Merkmal,Merkmalwert,Bestätigte Menge,Relativer Anteil,USTR10Y,WeizenSpot,CornSpot,GER10Y,WtiOilSpot,SoySpot,AgriSpot,Jahr,Monat,Monat_sin,Monat_cos
0,2014-01-01,B10,30,0,0.000000,3.006,197.738,187.924,1.941,98.04,246.732,200.399,2014,1,5.000000e-01,8.660254e-01
1,2014-02-01,B10,30,0,0.000000,2.582,182.795,190.254,1.647,96.43,246.293,201.583,2014,2,8.660254e-01,5.000000e-01
2,2014-03-01,B10,30,0,0.000000,2.608,204.039,200.847,1.563,104.92,269.561,221.079,2014,3,1.000000e+00,6.123234e-17
3,2014-04-01,B10,30,48,0.125654,2.755,222.170,215.466,1.581,99.74,281.258,233.772,2014,4,8.660254e-01,-5.000000e-01
4,2014-05-01,B10,30,42,0.156134,2.613,241.438,212.924,1.470,99.42,242.195,236.510,2014,5,5.000000e-01,-8.660254e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17552,2023-10-01,P02,2078,0,0.000000,4.687,198.708,217.797,2.911,88.82,260.098,234.182,2023,10,-8.660254e-01,5.000000e-01
17553,2023-11-01,P02,2078,10,0.033333,4.761,196.365,214.936,2.758,81.04,263.268,234.431,2023,11,-5.000000e-01,8.660254e-01
17554,2023-12-01,P02,2078,1,0.003846,4.220,199.354,215.360,2.363,73.72,266.732,232.888,2023,12,-2.449294e-16,1.000000e+00
17555,2024-01-01,P02,2078,0,0.000000,3.866,206.624,210.593,2.029,71.89,256.195,228.351,2024,1,5.000000e-01,8.660254e-01


## Train Test Split

Split data and only use training data for training the models!